In [2]:
import shioaji as sj
import pysimulation

api = sj.Shioaji(simulation=True)

accounts = api.login(
    api_key="3WmNiNGCQMuPGsRMdpgML3MWYkqM1gExzPUGKKcjhufd",     # 請修改此處
    secret_key="EnKAve1Ldd8wuerMRcwKUUrB4DRmLyoxCSyoApmbzrTM"   # 請修改此處
)

order = pysimulation.order('108062313') # 請改成自己的學號

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [3]:
# 近月臺股期貨商品檔
contract = min(
    [
        x for x in api.Contracts.Futures.TXF 
        if x.code[-2:] not in ["R1", "R2"]
    ],
    key=lambda x: x.delivery_date
)

from shioaji import TickFOPv1, BidAskFOPv1, Exchange

from collections import defaultdict, deque

msg_queue = defaultdict(deque)
api.set_context(msg_queue)

# In order to use context, set bind=True
@api.on_tick_fop_v1(bind=True)
def quote_callback(self, exchange:Exchange, tick:TickFOPv1):
    # append quote to message queue
    self['tick'].append(tick)

# In order to use context, set bind=True
@api.on_bidask_fop_v1(bind=True)
def quote_callback(self, exchange:Exchange, bidask:BidAskFOPv1):
    # append quote to message queue
    self['bidask'].append(bidask)

api.quote.subscribe(
    contract,
    quote_type = sj.constant.QuoteType.Tick,
    version = sj.constant.QuoteVersion.v1,
)

api.quote.subscribe(
    contract,
    quote_type = sj.constant.QuoteType.BidAsk,
    version = sj.constant.QuoteVersion.v1
)

Response Code: 200 | Event Code: 16 | Info: TIC/v1/FOP/*/TFE/TXFC3 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/FOP/*/TFE/TXFC3 | Event: Subscribe or Unsubscribe ok


In [4]:
import datetime
import pytz

while True:
#     print(datetime.datetime.now(pytz.timezone('ROC')).time())
    if datetime.datetime.now(pytz.timezone('ROC')).time() >= datetime.time(9, 15, 0):
        order.place_order(msg_queue['bidask'][-1], 'Buy', 'New')
        break
    else:
        continue
        
while True:
    if datetime.datetime.now(pytz.timezone('ROC')).time() >= datetime.time(13, 30, 0):
        if(len(order.list_trades()) != 0):
            if order.list_trades()[-1]['octype'] == 'New':
                if order.list_trades()[-1]['action'] == 'Buy':
                    # 當沖13:30強制沖銷，賣出平倉
                    order.place_order(msg_queue['bidask'][-1], 'Sell', 'Cover')
        break

KeyboardInterrupt: 

In [ ]:
# import datetime
# import pytz

# while True:
#     if datetime.datetime.now(pytz.timezone('ROC')).time() >= datetime.time(9, 15, 0):
#         high = msg_queue['tick'][-1]['high']
#         low = msg_queue['tick'][-1]['low']
#         # 開盤30分K，最高價最低價用於判斷多空
#         break
#     else:
#         continue

# while True:
#     while True:
# #         print(msg_queue['tick'])
#         if msg_queue['tick'][-1]['close'] > high:
#             # 成交價 > 最高價，新倉買進
#             order.place_order(msg_queue['bidask'][-1], 'Buy', 'New')
#             break
#         elif msg_queue['tick'][-1]['close'] < low:
#             # 成交價 < 最低價，新倉賣出
#             order.place_order(msg_queue['bidask'][-1], 'Sell', 'New')
#             break
#         elif datetime.datetime.now(pytz.timezone('ROC')).time() >= datetime.time(13, 30, 0):
#             break

#     while True:
#         # 2%停損
#         if datetime.datetime.now(pytz.timezone('ROC')).time() >= datetime.time(13, 30, 0):
#             break
#         else:
#             if order.list_trades()[-1]['action'] == 'Buy':
#                 if msg_queue['tick'][-1]['close'] < order.list_trades()[-1]['price'] * 0.98:
#                     # 新倉買進2%停損，賣出平倉
#                     order.place_order(msg_queue['bidask'][-1], 'Sell', 'Cover')
#                     break
#             elif order.list_trades()[-1]['action'] == 'Sell':
#                 if msg_queue['tick'][-1]['close'] > order.list_trades()[-1]['price'] * 1.02:
#                     # 新倉賣出2%停損，買進平倉
#                     order.place_order(msg_queue['bidask'][-1], 'Buy', 'Cover')
#                     break
    
#     if datetime.datetime.now(pytz.timezone('ROC')).time() >= datetime.time(13, 30, 0):
#         if(len(order.list_trades()) != 0):
#             if order.list_trades()[-1]['octype'] == 'New':
#                 if order.list_trades()[-1]['action'] == 'Buy':
#                     # 當沖13:30強制沖銷，賣出平倉
#                     order.place_order(msg_queue['bidask'][-1], 'Sell', 'Cover')
#                 elif order.list_trades()[-1]['action'] == 'Sell':
#                     # 當沖13:30強制沖銷，買進平倉
#                     order.place_order(msg_queue['bidask'][-1], 'Buy', 'Cover')
#         break

In [9]:
api.quote.unsubscribe(
    contract,
    quote_type = sj.constant.QuoteType.Tick,
    version = sj.constant.QuoteVersion.v1,
)

api.quote.unsubscribe(
    contract, 
    quote_type = sj.constant.QuoteType.BidAsk,
    version = sj.constant.QuoteVersion.v1
)


Response Code: 200 | Event Code: 16 | Info: TIC/v1/FOP/*/TFE/TXFC3 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/FOP/*/TFE/TXFC3 | Event: Subscribe or Unsubscribe ok


In [10]:
api.logout()

True

In [ ]:
# order.list_trades()